In [1]:
import requests
import json

# URL ENDPOINT
url = 'https://apps.distelsa.net/FlujoSku/Proveedores/cargarEMCs'

# CREDENCIALES COOKIES
login_data = {
    'txt_login_username': '11110',
    'txt_login_password': 'Chiquilla.21'
}


# DEFINICION DE HEADERS CON COOKIE ARRAFINITY Y ASP.NET_SESSIONID
headers = {
    'Cookie': 'ARRAffinity=c4ba1e2211e8bb24de1d78e35c68d1196e2f4c3e25e3040ec9f1c47f0412a550; ASP.NET_SessionId=gwxxpdbylqgdvn2fx4fo2zia',
    'Accept': 'application/json',
}

# PARAMETROS DE CONSULTA = ID DE SKU INTERNO DE ALEF
params = {
    'id': '1110',
}

# FUNCION QUOTEFIX
def quoteFix(original):
    if original is None:
        return None
    return original.replace('"', '&quot;')

# FUNCIONES DE ATRIBUTOS EMC
def emc_attr_string(attribute_id, default_frontend_label, valor_guardado):
    html_txt = ""
    html_txt += '<div class="uk-input-group">'
    html_txt += '    <span class="uk-input-group-addon"></span>'
    html_txt += '    <label for="' + attribute_id + ' ">' + default_frontend_label + '<span style="color:red;">*</span></label>'
    html_txt += '    <input type="text" class="md-input label-fixed" name="' + attribute_id + '" id="' + attribute_id + '" value="' + quoteFix(valor_guardado) + '"/>'
    html_txt += '</div>'
    return html_txt


def emc_att_decimal(attribute_id, default_frontend_label, valor_guardado):
    html_txt = ""
    html_txt += '<div class="uk-input-group">'
    html_txt += '    <span class="uk-input-group-addon"></span>'
    html_txt += '    <label for="' + attribute_id + ' ">' + default_frontend_label + '<span style="color:red;">*</span></label>'
    html_txt += '    <input type="number" min="0" onkeyup="if(this.value<0)this.value=0" onblur="if(this.value<0)this.value=0" class="md-input label-fixed" name="' + attribute_id + '" id="' + attribute_id + '" value="' + valor_guardado + '" step="any" />'
    html_txt += '</div>'
    return html_txt


def emc_attr_combo(attribute_id, default_frontend_label, valor_guardado, options):
    html_txt = ""
    html_txt += '<div class="uk-input-group uk-grid-margin">'
    html_txt += '   <span class="uk-input-group-addon"></span>'
    html_txt += '    <label for="' + attribute_id + '">' + default_frontend_label + '<span style="color:red;">*</span></label>'
    html_txt += '    <select id="' + attribute_id + '" name="' + attribute_id + '"data-md-selectize class="selectized">'
    html_txt += '        <option value="">Seleccione ...</option>'
    for option in options:
        is_selected = 'selected' if option['value'] == valor_guardado else ''
        html_txt += f'            <option {is_selected} value="{option["value"]}">{option["label"]}</option>'
    html_txt += '    </select>'
    html_txt += '</div>'
    return html_txt


# SOLICITUD GET CON URL HEADERS Y PARAMETROS CONFIGURADOS
response = requests.get(url, headers=headers, params=params)

# POSIBLES RESPUESTAS DEL ENDPOINT
if response.status_code == 200:
    print("------------------")
    print("SOLICITUD EXITOSA")
    print("------------------")
    data = response.json()
    
    print(data)
    #print(response.json())  # Asumiendo que la respuesta es JSON

    # Simulación de cómo se llamarían las funciones para generar el HTML
    # Aquí deberías manejar la estructura real de `data` para pasar los valores correctos
    emc_values = json.loads(data['EMC_values'])
    emcs = json.loads(data['EMCs'])[0]['attributes']
    #print(emc_values)
    fields_html = ""
    combos = []


    for emc in emcs:
        frontend_input = emc['frontend_input']
        attribute_code = emc['attribute_code']
        frontend_label = emc['frontend_label']
        valor_guardado = emc_values.get(attribute_code, "")
        print(frontend_label, valor_guardado)
        

        if frontend_input == "text":
            fields_html += emc_attr_string(attribute_code, frontend_label, valor_guardado)
        elif frontend_input in ["select", "boolean"]:
            fields_html += emc_attr_combo(attribute_code, frontend_label, valor_guardado, emc['options'])
            combos.append((attribute_code, emc['options']))  # Guardar el código y las opciones para imprimir después
        elif frontend_input in ["decimal", "price"]:
            fields_html += emc_att_decimal(attribute_code, frontend_label, valor_guardado)
    
    # Imprimir o usar fields_html según sea necesario
    #print(fields_html)
    
    # Imprimir los valores de cada combo
    for attribute_code, options in combos:
        print(f"GRUPO {attribute_code}:")
        for option in options:
            print(f" - VALUE: {option['value']}   LABEL: {option['label']}")
else:
    print(f"Error en la solicitud: {response.status_code}")

------------------
SOLICITUD EXITOSA
------------------
{'value': True, 'EMCs': '[{"attribute_group_id":"9134","attribute_group_name":"Especificaciones","attribute_group_code":"especificaciones","sort_order":"108","default_id":"0","tab_group_code":"basic","attributes":[{"attribute_id":"141","entity_type_id":"4","attribute_code":"tecnologia","attribute_model":null,"backend_model":null,"backend_type":"int","backend_table":null,"frontend_model":null,"frontend_input":"select","frontend_label":"Tecnolog\\u00eda","frontend_class":null,"source_model":"Magento\\\\Eav\\\\Model\\\\Entity\\\\Attribute\\\\Source\\\\Table","is_required":"1","is_user_defined":"1","default_value":"","is_unique":"0","note":null,"frontend_input_renderer":null,"is_global":"0","is_visible":"1","is_searchable":"1","is_filterable":"1","is_comparable":"1","is_visible_on_front":"1","is_html_allowed_on_front":"1","is_used_for_price_rules":"0","is_filterable_in_search":"0","used_in_product_listing":"1","used_for_sort_by":"0","

In [3]:
# Lista para almacenar los resultados
resultado_labels = []

# Iterar sobre los atributos y obtener los valores guardados correspondientes
for emc in emcs:
    attribute_code = emc['attribute_code']
    frontend_label = emc['frontend_label']
    valor_guardado = emc_values.get(attribute_code, "")
    
    # Si el input es de tipo combo, obtener el label correspondiente al valor guardado
    if emc['frontend_input'] in ["select", "boolean"]:
        valor_guardado_label = ""
        for option in emc['options']:
            if str(option['value']) == str(valor_guardado):  # Convertir ambos a string para comparación
                valor_guardado_label = option['label']
                break
        resultado_labels.append((frontend_label, valor_guardado_label))
    else:
        resultado_labels.append((frontend_label, valor_guardado))

# Mostrar los resultados
for label, valor in resultado_labels:
    print(f"{label}: {valor}")
    



Tecnología: LED
Tamaño de pantalla: 65"
Definición: Ultra High Definition 4K
Potencia de audio (Watts RMS): 20 W
Ancho: 146.3 cm
Alto sin base de mesa (cm): 85 cm
Alto con base de mesa (cm): 91.4 cm
Peso sin base de mesa (Kg): 21.3 Kg
Peso con base de mesa (Kg): 21.6 Kg
Sistema Operativo: Web-OS
Profundidad sin base de mesa (cm): 8.78 cm
HDR (High Dynamic Range): Yes
Smart TV / Acceso a Netflix y YouTube: Yes
Cantidad de Hertz: 60 Hz
Conexión USB: Yes
Conexiones HDMI / (ARC): 2
Conexión Wi-Fi: Si
Función espejo (Mirroring): Si
Conexión de audio / video (RCA): 0
Garantía: 2 años
Otras características: 
Conexión Bluetooth: Si
Salida de audio digital óptica: No
Profunidad con base de mesa (cm): 26.9 cm
Asistente de voz: ThinQ AI UHD
